In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Analyze collected data

## AMI NonVee

In [ ]:
#-------------------------
paths = Utilities.find_all_paths(
    base_dir=r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\AMINonVee', 
    glob_pattern=r'ami_nonvee_[0-9]*.csv', 
    regex_pattern=None
)
paths=natsorted(paths)
#-------------------------
outg_rec_nbs_in_files = dict()
for path in paths:
    assert(path not in outg_rec_nbs_in_files.keys())
    df = GenAn.read_df_from_csv(path)
    outg_rec_nbs_in_files[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
outg_rec_nb_to_files_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files)
all_outg_rec_nbs = list(outg_rec_nb_to_files_dict.keys())

In [ ]:
#-------------------------
paths_ede = Utilities.find_all_paths(
    base_dir=r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\EndEvents', 
    glob_pattern=r'end_events_[0-9]*.csv', 
    regex_pattern=None
)
paths_ede=natsorted(paths_ede)
#-------------------------
outg_rec_nbs_in_files_ede = dict()
for path in paths_ede:
    assert(path not in outg_rec_nbs_in_files_ede.keys())
    df = GenAn.read_df_from_csv(path)
    outg_rec_nbs_in_files_ede[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
outg_rec_nb_to_files_ede_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files_ede)
all_outg_rec_nbs_ede = list(outg_rec_nb_to_files_ede_dict.keys())

In [ ]:
outg_rec_nbs_in_files

In [ ]:
#-------------------------
instvabc_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value=['INSTVA1', 'INSTVB1', 'INSTVC1'], 
            comparison_operator='isin'
        )
    ], 
    name='VOLT, INSTV(ABC)1', 
    join_single_slicers='and'
)
#-------------------------
volt_avg_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value='AVG', 
            comparison_operator='=='
        )
    ], 
    name='VOLT, AVG', 
    join_single_slicers='and'
)
#-------------------------
slicers=[instvabc_slcr, volt_avg_slcr]

In [ ]:
#----------------------------------------------------------------------------------------------------
# Build mico_df_for_plt to be used in plotting times from mico_df
#----------------------------------------------------------------------------------------------------
conn_outages = Utilities.get_utldb01p_oracle_connection()
outgs_file_from_mico = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\2023-04-08 to 04-15 Reviews (1).xlsx'
expand_time = pd.Timedelta('1 day')
#----------------------------------------------------------------------------------------------------
mico_df_raw = pd.read_excel(outgs_file_from_mico, sheet_name='Scorecard')
mico_df = mico_df_raw.copy()
#-----
# For now, keep only the following columns:
mico_cols_to_keep = [
    'Outage #', 
    'Outage Start DT', 
    'Adj Outage Start DT', 
    'Outage End DT',
    'Adj Outage End DT', 
    'Circuit Name',
    'Step CMI'
]
mico_df = mico_df[mico_cols_to_keep]

#-------------------------
# Currently, outage numbers have -1, -2, etc. appended.
# I believe an outage number will have such multiple rows when the outage affects more than one circuit.
# In the DOVS database, these will be split iunto separate outg_rec_nbs
#-----
# I will instead merge the data via the outage number and circuit name, so remove the -1, -2, etc. from 
#   the 'Outage #', store the result in 'OUTAGE_NB' (to be consistent with DOVS), and drop 'Outage #'
mico_df['OUTAGE_NB'] = mico_df['Outage #'].apply(lambda x: re.sub('(\d*)-\d*', r'\1', x))
mico_df=mico_df.drop(columns=['Outage #'])

#-------------------------
# Each outage can also have multiple rows corresponding to the power recover steps
# Aggregate the steps into a single row for each outage
mico_df = mico_df.groupby(
    ['OUTAGE_NB', 'Circuit Name'], 
    dropna=False, 
    as_index=False, 
    group_keys=False
).agg({
    'Outage Start DT':     'min', 
    'Adj Outage Start DT': 'min', 
    'Outage End DT':       'max', 
    'Adj Outage End DT':   'max', 
    'Step CMI':            'sum'
})

# At this point, each outage (unique combinatino of 'OUTAGE_NB' and 'Circuit Name') should
#   correspond to a single row
assert(mico_df.shape[0] == mico_df.groupby(['OUTAGE_NB', 'Circuit Name']).ngroups)

#-------------------------
mico_df['Min Start Date'] = mico_df[['Outage Start DT', 'Adj Outage Start DT']].min(axis=1).dt.date - expand_time
mico_df['Max End Date']   = mico_df[['Outage End DT',   'Adj Outage End DT'  ]].max(axis=1).dt.date + expand_time

#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------

start = time.time()

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: A single OUTAGE_NB can correspond to more than one OUTG_REC_NBs!
#       It appears this is the case when the outage affects multiple GIS_CRCT_NBs, in which case,
#         each GIS_CRCT_NB gets its own OUTG_REC_NB
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# First, grab DF containing all OUTAGE_NBs
# Note, the OUTAGE_NB is not unique, so there will generally be multiple entries
#   Here, I'm talking about the same OUTAGE_NB being used for different outages throughout the years, 
#     not a single OUTAGE_NB corresponding to multiple OUTG_REC_NBs, as described above.
#   Determine which is correct entry using the times from mico_df
sql_using_outage_nbs = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    outage_nbs=mico_df['OUTAGE_NB'].unique().tolist(), 
    date_range=[mico_df['Min Start Date'].min(), mico_df['Max End Date'].max()], 
    MJR_CAUSE_CD=None, 
    DEVICE_CD=None, 
    INTRPTN_TYP_CD=None, 
    CURR_REC_STAT_CD=None, 
    select_cols_DOVS_PREMISE_DIM=['CIRCT_NM']
).get_sql_statement()
#-----
df_using_outage_nbs = pd.read_sql_query(
    sql_using_outage_nbs, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-------------------------
# Determine appropriate OUTG_REC_NBs by using 'Min Start Date', 'Max End Date'
df_using_outage_nbs = pd.merge(
    df_using_outage_nbs, 
    mico_df[['OUTAGE_NB', 'Circuit Name', 'Min Start Date', 'Max End Date']], 
    left_on=['OUTAGE_NB', 'CIRCT_NM'], 
    right_on=['OUTAGE_NB', 'Circuit Name'], 
    how='inner'
)
df_using_outage_nbs= df_using_outage_nbs[
    (df_using_outage_nbs['DT_OFF_TS_FULL'].dt.date >= df_using_outage_nbs['Min Start Date']) & 
    (df_using_outage_nbs['DT_ON_TS'].dt.date       <= df_using_outage_nbs['Max End Date'])
]
df_using_outage_nbs = df_using_outage_nbs.drop(columns=['Min Start Date', 'Max End Date', 'Circuit Name'])

print(time.time()-start)

# NOTE: Keep CIRCT_NM (although it comes from DOVS_PREMISE_DIM), as this will be used to merge as there exist 
#         duplicate OUTAGE_NB entries in Mico's df which have different OUTG_REC_NBs if the outage affects multiple circuits
df_outage_noPNs = df_using_outage_nbs.drop(columns=['OFF_TM', 'REST_TM', 'PREMISE_NB']).drop_duplicates()

#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
mico_df_for_plt = pd.merge(
    mico_df, 
    df_outage_noPNs[['OUTAGE_NB', 'CIRCT_NM', 'OUTG_REC_NB']], 
    left_on=['OUTAGE_NB', 'Circuit Name'], 
    right_on=['OUTAGE_NB', 'CIRCT_NM'], 
    how='inner'
)
#-----
mico_df_for_plt['Final Outage Start'] = mico_df_for_plt['Adj Outage Start DT'].fillna(mico_df_for_plt['Outage Start DT'])
mico_df_for_plt['Final Outage End']   = mico_df_for_plt['Adj Outage End DT'].fillna(mico_df_for_plt['Outage End DT'])
mico_df_for_plt = mico_df_for_plt.drop(columns=[
    'Adj Outage Start DT', 'Outage Start DT', 
    'Adj Outage End DT',   'Outage End DT', 
    'Circuit Name', 
    'Step CMI', 'Min Start Date', 'Max End Date'
])
mico_df_for_plt['OUTG_REC_NB'] = mico_df_for_plt['OUTG_REC_NB'].astype(str)

In [ ]:
calculate_by_PN = True
# combine_by_PN_likeness_thresh = pd.Timedelta('1 minutes')
combine_by_PN_likeness_thresh = pd.Timedelta('15 minutes')

expand_outg_search_time_tight = pd.Timedelta('1 hours')
expand_outg_search_time_loose = pd.Timedelta('12 hours')
use_est_outg_times=False
use_full_ede_outgs=False
run_outg_inclusion_assessment=False
#-----
expand_outg_est_search_time = expand_outg_search_time_loose
if use_est_outg_times:
    expand_outg_search_time = expand_outg_search_time_tight
else:
    expand_outg_search_time = expand_outg_search_time_loose

In [ ]:
save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results'
pdf = PdfPages(os.path.join(save_dir, r'ResultsNEW_PN2_NEWEDEMETHOD.pdf'))

In [ ]:
# outg_rec_nbs_uiq = [
#     '13294548',
#     '13294660',
#     '13294879',
#     '13295201',
#     '13295264',
#     '13297352',
#     '13297819',
#     '13298920',
#     '13298928',
#     '13298955',
#     '13299354',
#     '13300546',
#     '13300325',
#     '13300583',
#     '13302073',
#     '13302188',
#     '13302515',
#     '13303019',
#     '13304108'
# ]
# save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results'
# pdf = PdfPages(os.path.join(save_dir, r'Results_UIQ.pdf'))

In [ ]:
# # Below are actually all entries with some sort of comment
# outg_rec_nbs_uiq = [
#     '13294548',
#     '13294660',
#     '13294714',
#     '13294761',
#     '13294879',
#     '13294840',
#     '13295201',
#     '13295078',
#     '13295264',
#     '13295522',
#     '13295395',
#     '13295459',
#     '13295530',
#     '13297352',
#     '13297586',
#     '13297819',
#     '13298920',
#     '13298928',
#     '13298955',
#     '13299354',
#     '13300028',
#     '13300335',
#     '13300546',
#     '13300325',
#     '13300583',
#     '13302048',
#     '13302078',
#     '13302073',
#     '13302082',
#     '13302198',
#     '13302188',
#     '13302515',
#     '13303019',
#     '13303187',
#     '13303801',
#     '13304108',
#     '13304254'
# ]
# save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results'
# pdf = PdfPages(os.path.join(save_dir, r'Results_wComment.pdf'))

In [ ]:
fig_num=0
all_detailed_summary_dfs=[]
ci_cmi_summary_df = pd.DataFrame(columns=[
    'outg_rec_nb', 
    'ci_dovs',  'ci_ami', 
    'cmi_dovs', 'cmi_ami'
])

#-------------------------
# Build dovs_df
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=all_outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_premise=True
    ), 
    build_consolidated=True
)
dovs_df = dovs.df.copy()

#-------------------------
# Now, iterate through all outages
for i_outg, outg_rec_nb in enumerate(all_outg_rec_nbs):
    print(f'\n\ti_outg: {i_outg+1}/{len(all_outg_rec_nbs)}')
    print(f'\toutg_rec_nb = {outg_rec_nb}')
    #-----
    ami_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_dict[outg_rec_nb])
    #--------------------------------------------------
    ami_df_i = ami_df[ami_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()
    
    # Although I cannot yet call choose_best_slicer_and_perform_slicing and reduce_INSTV_ABC_1_vals_in_df, 
    #   as the standard cleaning and conversions must be done first, I am able to cut down the size of
    #   ami_df_i by joining the slicers with 'or' statements.
    # Thus, ami_df_i will be reduced to only the subset of data which will be considered in 
    #   choose_best_slicer_and_perform_slicing
    # As mentioned, this will cut down the size of ami_df_i and will also save time and resources by not having
    #   to run entire DF through cleaning and conversions procedures.
    ami_df_i = DFSlicer.combine_slicers_and_perform_slicing(
        df=ami_df_i, 
        slicers=slicers, 
        join_slicers='or'
    )
    if ami_df.shape[0]==0:
        continue
        
    #--------------------------------------------------
    ami_df_i = AMINonVee.perform_std_initiation_and_cleaning(ami_df_i)
    #-----
    # Should the following be added to AMINonVee.perform_std_initiation_and_cleaning?
    ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df_i, 
        time_col='starttimeperiod', 
        placement_col='starttimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df_i, 
        time_col='endtimeperiod', 
        placement_col='endtimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    #--------------------------------------------------
    ami_df_i = choose_best_slicer_and_perform_slicing(
        df=ami_df_i, 
        slicers=slicers, 
        groupby_SN=True, 
        t_search_min_max=None, 
        time_col='starttimeperiod_local', 
        value_col=None, 
        SN_col='serialnumber', 
        return_sorted=True
    )

    ami_df_i = reduce_INSTV_ABC_1_vals_in_df(
        df=ami_df_i, 
        value_col='value', 
        aep_derived_uom_col='aep_derived_uom', 
        aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
        output_aep_srvc_qlty_idntfr = 'INSTV(ABC)1'
    )

    if ami_df_i.shape[0]==0:
        continue
        
    #-------------------------
    # Each serial number should have a single value per time stamp
    assert(ami_df_i.groupby(['serialnumber', 'starttimeperiod_local']).ngroups == ami_df_i.shape[0])

    #-------------------------
    if run_outg_inclusion_assessment:
        to_include_i = assess_outage_inclusion_requirements(
            ami_df_i=ami_df_i, 
            outg_rec_nb=outg_rec_nb, 
            dovs_df=dovs_df, 
            max_pct_PNs_missing_allowed=0

        )
        if not to_include_i:
            print(f'outg_rec_nb={outg_rec_nb} did not pass inclusion requirements, skipping!!!!!')
            continue
    #-------------------------    
    n_SNs  = ami_df_i['serialnumber'].nunique()
    n_PNs  = ami_df_i['aep_premise_nb'].nunique()

    #----------------------------------------------------------------------------------------------------
    # NOTE: Can save time by grabbing ede_df_i then performing tz conversion and adding DOVS
    if outg_rec_nb not in outg_rec_nb_to_files_ede_dict.keys():
        ede_df_i=None
    else:
        ede_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_ede_dict[outg_rec_nb])
        ede_df_i = ede_df[ede_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()

        #-----
        ede_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=ede_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        ede_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(df=ede_df_i)
        #-----
        ede_cols_to_keep = [
            'valuesinterval_local', 
            'reason', 
            'serialnumber', 
            'aep_premise_nb', 
            'enddeviceeventtypeid', 
            'event_type', 
            'OUTG_REC_NB_GPD_FOR_SQL', 
            'trsf_pole_nb_GPD_FOR_SQL',
        ]
        ede_df_i = ede_df_i[ede_cols_to_keep]

    #----------------------------------------------------------------------------------------------------
    dovs_df_i = DOVSOutages.retrieve_outage_from_dovs_df(
        dovs_df=dovs_df, 
        outg_rec_nb=outg_rec_nb, 
        outg_rec_nb_idfr='index', 
        assert_outg_rec_nb_found=True
    )
    assert(dovs_df_i.shape[0]==1)
    # Get the outage time from DOVS
    dovs_outg_t_beg_end = dovs_df_i.iloc[0][['DT_OFF_TS_FULL', 'DT_ON_TS']].tolist()
    assert(len(dovs_outg_t_beg_end)==2)
    dovs_outg_t_beg, dovs_outg_t_end = dovs_outg_t_beg_end
    #-------------------------
    # Get the CI and CMI from DOVS
    ci_cmi_dovs = dovs_df_i.iloc[0][['CI_NB', 'CMI_NB']].tolist()
    assert(len(ci_cmi_dovs)==2)
    ci_dovs, cmi_dovs = ci_cmi_dovs
    #-------------------------
    # Get the number of premises from DOVS
    n_PNs_dovs = len(set(dovs_df_i.iloc[0]['premise_nbs']))
    #-------------------------
    # Get the outage number from DOVS
    outage_nb = dovs_df_i.iloc[0]['OUTAGE_NB']

    #--------------------------------------------------
    res_dict = calculate_ci_cmi_w_ami_w_ede_help(
        df=ami_df_i, 
        ede_df=ede_df_i, 
        dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
        expand_outg_search_time=expand_outg_search_time, 
        conservative_estimate=True, 
        est_ede_kwargs=dict(use_full_ede_outgs=use_full_ede_outgs), 
        audit_selection_method='ede only', 
        return_CI_SNs=False, 
        use_est_outg_times=use_est_outg_times, 
        pct_SNs_required_for_outage_est=0, 
        expand_outg_est_search_time=expand_outg_est_search_time, 
        use_only_overall_endpoints_of_est_outg_times=False, 
        time_col='starttimeperiod_local', 
        value_col='value', 
        SN_col='serialnumber', 
        return_all_best_ests=True, 
        return_all_best_ests_type='pd.DataFrame'
    )
    #-----
    ci_ami       = res_dict['CI']
    cmi_ami      = res_dict['CMI']
    best_ests_df = res_dict['all_best_ests']
    #--------------------------------------------------
    if calculate_by_PN and best_ests_df.shape[0]>0:
        best_ests_df = combine_PNs_in_best_ests_df(
            best_ests_df, 
            likeness_thresh = combine_by_PN_likeness_thresh, 
            SN_col = 'SN', 
            PN_col = 'PN', 
            i_outg_col = 'i_outg'     
        )
        ci_ami  = best_ests_df['PN'].nunique()
        cmi_ami = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
    #--------------------------------------------------
    if best_ests_df.shape[0]>0:
        means_df, best_ests_df_w_db_lbl = get_mean_times_w_dbscan(
            best_ests_df, 
            eps_min=5, 
            min_samples=2, 
            ests_to_include_in_clustering=['winner_min', 'winner_max'],
            ests_to_include_in_output=[
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df=True
        )
        #-----
        detailed_summary_df_i = build_detailed_summary_df(
            means_df=means_df, 
            best_ests_df_w_db_lbl=best_ests_df_w_db_lbl,
            CI_tot=ci_ami, 
            CMI_tot=cmi_ami, 
            n_PNs_ami=n_PNs,
            outg_rec_nb=outg_rec_nb, 
            dovs_df_i=dovs_df_i, 
            db_label_col='db_label', 
            winner_min_col='winner_min', 
            winner_max_col='winner_max', 
            PN_col='PN' if calculate_by_PN else 'SN', 
            i_outg_col='i_outg'
        )
        all_detailed_summary_dfs.append(detailed_summary_df_i)
    else:
        means_df, best_ests_df_w_db_lbl = None, None
    #-------------------------
    ci_cmi_summary_df = pd.concat([
        ci_cmi_summary_df, 
        pd.DataFrame(
            dict(
                outg_rec_nb=outg_rec_nb, 
                ci_dovs=ci_dovs,   ci_ami=ci_ami, 
                cmi_dovs=cmi_dovs, cmi_ami=cmi_ami
            ), 
            index=[ci_cmi_summary_df.shape[0]]
        )
    ])
    #----------------------------------------------------------------------------------------------------
    # ######################### PLOTTING #########################
    #----------------------------------------------------------------------------------------------------
    if means_df is not None:
        cnsrvtv_out_t_beg = np.min([means_df.min().min(), dovs_outg_t_beg])
        cnsrvtv_out_t_end = np.max([means_df.max().max(), dovs_outg_t_end])
    else:
        cnsrvtv_out_t_beg = dovs_outg_t_beg
        cnsrvtv_out_t_end = dovs_outg_t_end
    #--------------------------------------------------
    # Instead of using get_full_part_not_outage_subset_dfs, simply grab the PNs which suffered
    #   outages from best_ests_df
    if best_ests_df.shape[0]>0:
        outg_SNs = best_ests_df['PN'].unique().tolist()
    else:
        outg_SNs = []
    #-----
    ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
    ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]  
    
    #----------------------------------------------------------------------------------------------------
    fig, axs = Plot_General.default_subplots(
        n_x=1,
        n_y=3,
        fig_num=fig_num,
        sharex=False,
        sharey=False,
        unit_figsize_width=14,
        unit_figsize_height=6, 
        return_flattened_axes=True,
        row_major=True
    )
    Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

    palette = Plot_General.get_standard_colors_dict(
        keys=ami_df_i['serialnumber'].unique().tolist(), 
        palette='colorblind'
    )

    #-------------------------
    i_subplot=0
    fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
        fig=fig, 
        ax=axs[i_subplot], 
        data=ami_df_i, 
        x='starttimeperiod_local', 
        y='value', 
        hue='serialnumber', 
        out_t_beg=dovs_outg_t_beg, 
        out_t_end=dovs_outg_t_end, 
        expand_time=pd.Timedelta('1 hour'), 
        plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
        data_label='', 
        title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
        ax_args=None, 
        xlabel_args=None, 
        ylabel_args=None, 
        df_mean=None, 
        df_mean_col=None, 
        mean_args=None, 
        draw_outage_limits=True, 
        draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
        include_outage_limits_text=dict(
            out_t_beg_text='DOVS Beg.', 
            out_t_beg_ypos=(0.12, 'ax_coord'), 
            out_t_beg_va='bottom', 
            out_t_beg_ha='center', 
            out_t_beg_color='red', 
            #-----
            out_t_end_text='DOVS End', 
            out_t_end_ypos=(0.12, 'ax_coord'), 
            out_t_end_va='bottom', 
            out_t_end_ha='center', 
            out_t_end_color='green', 
        ), 
        draw_without_hue_also=False, 
        seg_line_freq=None, 
        palette=palette
    )
    axs[i_subplot].legend().set_visible(False)
    Plot_General.set_general_plotting_args(
        ax=axs[i_subplot], 
        tick_args =[
            dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
            dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
        ], 
        xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
        ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
    )


    #-------------------------
    i_subplot=1
    if ami_df_i_out.shape[0]>0:
        fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_subplot], 
            data=ami_df_i_out, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=pd.Timedelta('1 hour'), 
            plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
            data_label='', 
            title_args=dict(label=f"Out (#SNs = {ami_df_i_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ), 
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_subplot].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_subplot], 
            means_df, 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=['winner', 'conservative', 'zero_times']
        )

        #-------------------------
        if mico_df_for_plt is None:
            include_mico_in_leg = False
        else:
            mico_df_for_plt_i = mico_df_for_plt[
                (mico_df_for_plt['OUTAGE_NB']==dovs_df.loc[outg_rec_nb]['OUTAGE_NB']) & 
                (mico_df_for_plt['OUTG_REC_NB']==outg_rec_nb)
            ]
            if mico_df_for_plt_i.shape[0]>0:
                assert(mico_df_for_plt_i.shape[0]==1)
                include_mico_in_leg = True
                mico_beg, mico_end = mico_df_for_plt_i.iloc[0][['Final Outage Start', 'Final Outage End']]
                add_best_est_to_axis(
                    ax=axs[i_subplot], 
                    est_val_beg=mico_beg,
                    est_val_end=mico_end,
                    line_kwargs=dict(color_beg='maroon', color_end='darkgreen', linestyle='dotted'), 
                    expand_ax_to_accommodate=False
                )
            else:
                include_mico_in_leg = False
        #-------------------------

        Plot_General.set_general_plotting_args(
            ax=axs[i_subplot], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
                dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
            ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
        )
    else:
        axs[i_subplot].set_title(
            label=f'Out', 
            fontdict=dict(fontsize=24)
        )

    #-------------------------
    i_subplot=2
    if ami_df_i_not_out.shape[0]>0:
        fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_subplot], 
            data=ami_df_i_not_out, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=pd.Timedelta('1 hour'), 
            plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
            data_label='', 
            title_args=dict(label=f"Not Out (#SNs = {ami_df_i_not_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ), 
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_subplot].legend().set_visible(False)
        Plot_General.set_general_plotting_args(
            ax=axs[i_subplot], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
                dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
            ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
        )
    else:
        axs[i_subplot].set_title(label='Not Out', fontdict=dict(fontsize=24))


    #--------------------------------------------------
    # Add legend to first plot
    patch_dovs_beg = Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS Beg.'
    )
    patch_dovs_end = Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS End'
    )
    #-----
    patch_ui_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='Beg. Uncertainty Interval'
    )
    patch_ui_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='End Uncertainty Interval'
    )
    #-----
    patch_best_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. Beg.'
    )
    patch_best_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. End'
    )
    #-------------------------
    handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end]
    if include_mico_in_leg:
        patch_mico_beg =  Line2D(
            [0], [0], color='maroon', 
            alpha=1.0, linewidth=1.0, linestyle='dotted', 
            label='Mico Xlsx Beg.'
        )
        patch_mico_end =  Line2D(
            [0], [0], color='darkgreen', 
            alpha=1.0, linewidth=1.0, linestyle='dotted', 
            label='Mico Xlsx End'
        )
        handles.extend([patch_mico_beg, patch_mico_end])
    #-------------------------
    leg_1 = axs[0].legend(
        title=None, 
        handles=handles, 
        bbox_to_anchor=(1, 1.025), 
        loc='upper left', 
        fontsize=15
    )        

    #--------------------------------------------------
    ci_info_fontsize = 20
    left_text_x=0.95
    right_text_x = 1.05
    if include_mico_in_leg:
        shift_text_down = 0.05
    else:
        shift_text_down = 0

    fig.text(left_text_x, 0.745-shift_text_down, f'OUTG_REC_NB: {outg_rec_nb}', fontsize=ci_info_fontsize+4)
    fig.text(left_text_x, 0.715-shift_text_down, f"OUTAGE_NB:     {dovs_df.loc[outg_rec_nb]['OUTAGE_NB']}", fontsize=ci_info_fontsize+4)

    fig.text(left_text_x, 0.675-shift_text_down, f"#PNs from DOVS = {n_PNs_dovs}", fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.640-shift_text_down, "----- Found in AMI -----", fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.615-shift_text_down, f"#PNs = {n_PNs}", fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.590-shift_text_down, f"#SNs = {n_SNs}", fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.485-shift_text_down, '-----'*5+'\nDOVS\n'+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.460-shift_text_down, f'CI    = {ci_dovs}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.435-shift_text_down, f'CMI = {np.round(cmi_dovs, decimals=2)}', fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.360-shift_text_down, '-----'*5+'\nUsing AMI\n'+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.335-shift_text_down, f'CI    = {ci_ami}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.310-shift_text_down, f'CMI = {np.round(cmi_ami, decimals=2)}', fontsize=ci_info_fontsize)
    #-----
    fig.text(
        left_text_x, 0.285-shift_text_down, 
        f'$\Delta$CI    = {ci_dovs-ci_ami} ({np.round(100*(ci_dovs-ci_ami)/ci_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )
    fig.text(
        left_text_x, 0.260-shift_text_down, 
        f'$\Delta$CMI = {np.round(cmi_dovs-cmi_ami, decimals=2)} ({np.round(100*(cmi_dovs-cmi_ami)/cmi_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )

    fig_num += 1
    pdf.savefig(fig, bbox_inches='tight')
    plt.close(fig)
#----------------------------------------------------------------------------------------------------
detailed_summary_df = pd.concat(all_detailed_summary_dfs)

In [ ]:
pdf.close()
detailed_summary_df.to_pickle(os.path.join(save_dir, r'SummaryNEW_PN2_NEWEDEMETHOD.pkl'))
#-----
# detailed_summary_df.to_csv(os.path.join(save_dir, r'SummaryNEW.csv'), index=True)
# #-----
# detailed_summary_df_for_csv = drop_multiindex_duplicate_idxs_for_csv_output(detailed_summary_df.copy())
# detailed_summary_df_for_csv.to_csv(os.path.join(save_dir, r'Summary_dropMultiNEW.csv'), index=False)

In [ ]:
ci_cmi_summary_df['ci_dovs']   = ci_cmi_summary_df['ci_dovs'].astype(float)
ci_cmi_summary_df['ci_ami'] = ci_cmi_summary_df['ci_ami'].astype(float)
#-----
ci_cmi_summary_df['delta_ci_dovs_ami']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami']
ci_cmi_summary_df['delta_cmi_dovs_ami'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami']
#-----
# For plotting purposes, make a outg_rec_in column which is simply 0 to delta_df.shape[0]-1
ci_cmi_summary_df['outg_rec_int'] = range(ci_cmi_summary_df.shape[0])
#-----
ci_cmi_summary_df.to_pickle(os.path.join(save_dir, r'ci_cmi_summary.pkl'))

In [ ]:
assert(0)

In [ ]:
summary_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\Summary.pkl')
summary_df = Utilities_df.convert_col_types(summary_df, {'outg_i_beg':datetime.datetime, 'outg_i_end':datetime.datetime})

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()

In [ ]:
outgs_file_from_mico = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\2023-04-08 to 04-15 Reviews (1).xlsx'
expand_time = pd.Timedelta('1 day')

In [ ]:
#-------------------------
mico_df_raw = pd.read_excel(outgs_file_from_mico, sheet_name='Scorecard')
mico_df = mico_df_raw.copy()
#-----
# For now, keep only the following columns:
mico_cols_to_keep = [
    'Outage #', 
    'Outage Start DT', 
    'Adj Outage Start DT', 
    'Outage End DT',
    'Adj Outage End DT', 
    'Circuit Name',
    'Step CMI', 
    
    'Count of AMI Flag',
    'Count of AMI Y/N',
    'Min of Outage % AMI', 
    
    'Analyst Comments'
]
mico_df = mico_df[mico_cols_to_keep]

# NOTE: Although 'Min of Outage % AMI' column appears on range 0-100% in Excel, when read in as csv,
#       it is converted to range 0-1
mico_df['Min of Outage % AMI'] = 100.*mico_df['Min of Outage % AMI']

#-------------------------
# Currently, outage numbers have -1, -2, etc. appended.
# I believe an outage number will have such multiple rows when the outage affects more than one circuit.
# In the DOVS database, these will be split iunto separate outg_rec_nbs
#-----
# I will instead merge the data via the outage number and circuit name, so remove the -1, -2, etc. from 
#   the 'Outage #', store the result in 'OUTAGE_NB' (to be consistent with DOVS), and drop 'Outage #'
mico_df['OUTAGE_NB'] = mico_df['Outage #'].apply(lambda x: re.sub('(\d*)-\d*', r'\1', x))
mico_df=mico_df.drop(columns=['Outage #'])

#-------------------------
# Each outage can also have multiple rows corresponding to the power recover steps
# Aggregate the steps into a single row for each outage
mico_df = mico_df.groupby(
    ['OUTAGE_NB', 'Circuit Name'], 
    dropna=False, 
    as_index=False, 
    group_keys=False
).agg({
    'Outage Start DT':     'min', 
    'Adj Outage Start DT': 'min', 
    'Outage End DT':       'max', 
    'Adj Outage End DT':   'max', 
    'Step CMI':            'sum', 
    
    'Count of AMI Flag':   'first',
    'Count of AMI Y/N':    'first',
    'Min of Outage % AMI': 'first', 
    
    'Analyst Comments': 'first'
})

# At this point, each outage (unique combinatino of 'OUTAGE_NB' and 'Circuit Name') should
#   correspond to a single row
assert(mico_df.shape[0] == mico_df.groupby(['OUTAGE_NB', 'Circuit Name']).ngroups)

#-------------------------
mico_df['Min Start Date'] = mico_df[['Outage Start DT', 'Adj Outage Start DT']].min(axis=1).dt.date - expand_time
mico_df['Max End Date']   = mico_df[['Outage End DT',   'Adj Outage End DT'  ]].max(axis=1).dt.date + expand_time

In [ ]:
mico_df['OUTAGE_NB'].nunique()

In [ ]:
mico_df

In [ ]:
start = time.time()

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: A single OUTAGE_NB can correspond to more than one OUTG_REC_NBs!
#       It appears this is the case when the outage affects multiple GIS_CRCT_NBs, in which case,
#         each GIS_CRCT_NB gets its own OUTG_REC_NB
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# First, grab DF containing all OUTAGE_NBs
# Note, the OUTAGE_NB is not unique, so there will generally be multiple entries
#   Here, I'm talking about the same OUTAGE_NB being used for different outages throughout the years, 
#     not a single OUTAGE_NB corresponding to multiple OUTG_REC_NBs, as described above.
#   Determine which is correct entry using the times from mico_df
sql_using_outage_nbs = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    outage_nbs=mico_df['OUTAGE_NB'].unique().tolist(), 
    date_range=[mico_df['Min Start Date'].min(), mico_df['Max End Date'].max()], 
    MJR_CAUSE_CD=None, 
    DEVICE_CD=None, 
    INTRPTN_TYP_CD=None, 
    CURR_REC_STAT_CD=None, 
    select_cols_DOVS_PREMISE_DIM=['CIRCT_NM']
).get_sql_statement()
#-----
df_using_outage_nbs = pd.read_sql_query(
    sql_using_outage_nbs, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-------------------------
# Determine appropriate OUTG_REC_NBs by using 'Min Start Date', 'Max End Date'
df_using_outage_nbs = pd.merge(
    df_using_outage_nbs, 
    mico_df[['OUTAGE_NB', 'Circuit Name', 'Min Start Date', 'Max End Date']], 
    left_on=['OUTAGE_NB', 'CIRCT_NM'], 
    right_on=['OUTAGE_NB', 'Circuit Name'], 
    how='inner'
)
df_using_outage_nbs= df_using_outage_nbs[
    (df_using_outage_nbs['DT_OFF_TS_FULL'].dt.date >= df_using_outage_nbs['Min Start Date']) & 
    (df_using_outage_nbs['DT_ON_TS'].dt.date       <= df_using_outage_nbs['Max End Date'])
]
df_using_outage_nbs = df_using_outage_nbs.drop(columns=['Min Start Date', 'Max End Date', 'Circuit Name'])

print(time.time()-start)

In [ ]:
df_outage = df_using_outage_nbs
outg_rec_nbs_all = df_outage['OUTG_REC_NB'].unique().tolist()
#-----
print(f"df_outage.shape = {df_outage.shape}")
print(f"# OUTG_REC_NBs  = {df_outage['OUTG_REC_NB'].nunique()}")

# NOTE: Keep CIRCT_NM (although it comes from DOVS_PREMISE_DIM), as this will be used to merge as there exist 
#         duplicate OUTAGE_NB entries in Mico's df which have different OUTG_REC_NBs if the outage affects multiple circuits
df_outage_noPNs = df_outage.drop(columns=['OFF_TM', 'REST_TM', 'PREMISE_NB']).drop_duplicates()

In [ ]:
summary_df = summary_df.reset_index()

In [ ]:
df_outage_noPNs.groupby(['OUTG_REC_NB', 'OUTAGE_NB']).ngroups

In [ ]:
summary_df.groupby(['OUTG_REC_NB', 'OUTAGE_NB']).ngroups

In [ ]:
summary_df_to_mrg = summary_df[summary_df['Outage Subset']=='Full Outage'].copy()

In [ ]:
df_outage_noPNs['OUTG_REC_NB'] = df_outage_noPNs['OUTG_REC_NB'].astype(str)
set(summary_df_to_mrg.groupby(['OUTAGE_NB', 'OUTG_REC_NB']).groups.keys()).difference(
    set(df_outage_noPNs.groupby(['OUTAGE_NB', 'OUTG_REC_NB']).groups.keys())
)

In [ ]:
summary_df_to_mrg=summary_df_to_mrg.merge(
    df_outage_noPNs[['OUTAGE_NB', 'OUTG_REC_NB', 'CIRCT_NM', 'DT_OFF_TS_FULL', 'DT_ON_TS']], 
    left_on=['OUTAGE_NB', 'OUTG_REC_NB'], 
    right_on=['OUTAGE_NB', 'OUTG_REC_NB'], 
    how='left'
)

In [ ]:
summary_df_to_mrg=summary_df_to_mrg.merge(
    mico_df, 
    left_on=['OUTAGE_NB', 'CIRCT_NM'], 
    right_on=['OUTAGE_NB', 'Circuit Name'], 
    how='left'
)

In [ ]:
summary_df_to_mrg['abs(Delta Start) (min.)'] = summary_df_to_mrg['Adj Outage Start DT'].fillna(summary_df_to_mrg['Outage Start DT']) - summary_df_to_mrg['outg_i_beg']
summary_df_to_mrg['abs(Delta Start) (min.)'] = summary_df_to_mrg['abs(Delta Start) (min.)'].apply(lambda x: np.abs(np.round(x.total_seconds()/60, decimals=2)))
#-----
summary_df_to_mrg['abs(Delta End) (min.)'] = summary_df_to_mrg['Adj Outage End DT'].fillna(summary_df_to_mrg['Outage End DT']) - summary_df_to_mrg['outg_i_end']
summary_df_to_mrg['abs(Delta End) (min.)'] = summary_df_to_mrg['abs(Delta End) (min.)'].apply(lambda x: np.abs(np.round(x.total_seconds()/60, decimals=2)))

In [ ]:
summary_df_to_mrg_cols_order = [
    'OUTAGE_NB',
    'OUTG_REC_NB',
    'Circuit Name', 
    
    'DT_OFF_TS_FULL', 
    'Outage Start DT',
    'Adj Outage Start DT',
    'outg_i_beg',
    'abs(Delta Start) (min.)', 
    
    'DT_ON_TS', 
    'Outage End DT',
    'Adj Outage End DT', 
    'outg_i_end',
    'abs(Delta End) (min.)', 
    
    'Count of AMI Flag',
    'n_PNs_ami',
    
    'Count of AMI Y/N',
    'n_PNs_dovs',
    
    'Min of Outage % AMI',
    'pct_PNs_found',
    
    'Step CMI',
    'CMI_i',
    'CI_i',
    
    'Analyst Comments'
]
summary_df_to_mrg = summary_df_to_mrg[summary_df_to_mrg_cols_order]

In [ ]:
summary_df_to_mrg

In [ ]:
summary_df_to_mrg=summary_df_to_mrg.rename(columns={
    'DT_OFF_TS_FULL': 'Outage Start DT (DOVS)', 
    'outg_i_beg':     'Outage Start DT (JB)', 
    
    'DT_ON_TS':   'Outage End DT (DOVS)', 
    'outg_i_end': 'Outage End DT (JB)', 
    
    'n_PNs_ami':     'Count AMI PNs used (JB)', 
    'n_PNs_dovs':    'Count PNs DOVS', 
    'pct_PNs_found': '% DOVS PNs Used', 
    
    'Step CMI': 'SUM(Step CMI)', 
    
    'CMI_i': 'CMI (JB)', 
    'CI_i':  'CI (JB)'
})

In [ ]:
# summary_df_to_mrg.to_csv(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\Summary_mergeMicoXlsx_v0.csv', index=False)

In [ ]:
summary_df_to_mrg_w_empty_cols = summary_df_to_mrg.copy()

In [ ]:
# Empty columns will be placed before these
cols_beg_gps = [
    'Outage Start DT (DOVS)', 
    'Outage End DT (DOVS)', 
    'Count of AMI Flag', 
    'SUM(Step CMI)'
]

In [ ]:
for i, col in enumerate(cols_beg_gps): 
    summary_df_to_mrg_w_empty_cols.insert(
        loc=Utilities_df.find_idxs_in_highest_order_of_columns(summary_df_to_mrg_w_empty_cols, col)[0], 
        #column=f'SPACER {(i+1)}', 
        column=' '*20+' '*i, 
        value=['']*len(summary_df_to_mrg_w_empty_cols)
    )

In [ ]:
summary_df_to_mrg_w_empty_cols.columns.tolist()

In [ ]:
summary_df_to_mrg_w_empty_cols

In [ ]:
# summary_df_to_mrg_w_empty_cols.to_csv(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\Summary_mergeMicoXlsx.csv', index=False)

In [ ]:
# summary_df_to_mrg_w_empty_cols[
#     summary_df_to_mrg_w_empty_cols['Analyst Comments']=='UIQ'
# ].to_csv(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\Summary_mergeMicoXlsx_UIQ.csv', index=False)

In [ ]:
# summary_df_to_mrg_w_empty_cols[
#     summary_df_to_mrg_w_empty_cols['Analyst Comments']=='UIQ'
# ]['OUTG_REC_NB'].unique().tolist()

In [ ]:
# summary_df_to_mrg_w_empty_cols[
#     summary_df_to_mrg_w_empty_cols['Analyst Comments'].notna()
# ].to_csv(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\Summary_mergeMicoXlsx_wComment.csv', index=False)

In [ ]:
# summary_df_to_mrg_w_empty_cols[
#     summary_df_to_mrg_w_empty_cols['Analyst Comments'].notna()
# ]['OUTG_REC_NB'].unique().tolist()

In [ ]:
df1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\SummaryNEW_PN2.pkl')
df2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2\Results\SummaryNEW_PN2_NEWEDEMETHOD.pkl')

In [ ]:
df1.equals(df2)

In [ ]:
print(df1.shape)
print(df2.shape)

In [ ]:
set(df1.index).difference(set(df2.index))

In [ ]:
df1

In [ ]:
diffs = Utilities_df.get_dfs_diff(df1.reset_index(), df2.reset_index())
diffs

In [ ]:
df1

In [ ]:
diff_outg_rec_nbs = df2.loc[list(set(df2.index).difference(set(df1.index)))].reset_index()['OUTG_REC_NB'].unique().tolist()

In [ ]:
df2[df2.index.get_level_values(1).isin(diff_outg_rec_nbs)]

In [ ]:
df1[df1.index.get_level_values(1).isin(diff_outg_rec_nbs)]

In [ ]:
df_1_comp = df1.loc[list(set(df1.index).intersection(set(df2.index)))].copy().reset_index()
df_2_comp = df2.loc[list(set(df1.index).intersection(set(df2.index)))].copy().reset_index()

In [ ]:
df_1_comp

In [ ]:
df_2_comp

In [ ]:
diffs = Utilities_df.get_dfs_diff_approx_ok(df_1_comp, df_2_comp, precision=0.00001, return_df_only=True)
diffs

In [ ]:
df_1_comp.loc[20]

In [ ]:
ci_cmi_summary_df

In [ ]:
df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\Results_ede_only_v2.pkl')

In [ ]:
df